<a href="https://colab.research.google.com/github/learningdollars/reddit-twitter-convo-data/blob/master/Reddit_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complete Reddit Selenum & BeautifulSoup Scraping



## Preparation

In [12]:
# Install chromium, driver, and selenium
!pip install selenium
!pip install pandas
!pip install anytree
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Adding to path, unnecessary if files are moved to /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (78.0.3904.97-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


## Initialization

In [0]:
# Hyperparameters
PAGES = 3   # Number of pages from each subreddit
HTTPS = "https://old.reddit.com"
SUBREDDIT = [
             "r/all",
             "r/funny",
             "r/jokes"
             ]
SORT = ""

In [0]:
# Import libraries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as soup

import pandas as pd
import anytree
from anytree import Node, RenderTree
from pprint import pprint

# Initialize webdriver, set to headless
options = webdriver.ChromeOptions()
options.add_argument("-no-sandbox")
options.add_argument("-headless")
options.add_argument("-disable-dev-shm-usage")

# Open driver with a website, then get result
d = webdriver.Chrome("chromedriver", options=options)
d.get("https://old.reddit.com/r/GlobalOffensive/")
# print(d.page_source) # results

In [0]:
# # !pip install pandas
# import pandas as pd

# Scrape

In [16]:
# Get all posts
posts = []

for i in range(PAGES):
  wdposts = d.find_elements_by_class_name("thing")

  # Iterate through every post found
  for wdpost in wdposts:
    html = wdpost.get_attribute("outerHTML")
    attrs = soup(html, "html.parser").div.attrs
    posts.append(attrs)
  
  # Go to next page
  next = d.find_element_by_class_name("next-button").get_attribute("outerHTML")
  page = soup(next, "html.parser").a.attrs["href"]
  print(i, page)
  d.get(page)

# Create table using pandas DataFrame
dfposts = pd.DataFrame(posts)
dfposts[0:4] # Preview first 4 posts

0 https://old.reddit.com/r/GlobalOffensive/?count=25&after=t3_dyzrg5
1 https://old.reddit.com/r/GlobalOffensive/?count=50&after=t3_dyz76a
2 https://old.reddit.com/r/GlobalOffensive/?count=75&after=t3_dyq8sg


,class,id,onclick,data-fullname,data-type,data-gildings,data-whitelist-status,data-author,data-author-fullname,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-timestamp,data-url,data-permalink,data-domain,data-rank,data-comments-count,data-score,data-promoted,data-nsfw,data-spoiler,data-oc,data-num-crossposts,data-context,data-kind
0,"[, thing, id-t3_dybzmg, linkflair, linkflair-s...",thing_t3_dybzmg,click_thing(this),t3_dybzmg,link,0,all_ads,_Xertz,t2_126h0x,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1574121487000,/r/GlobalOffensive/comments/dybzmg/operation_s...,/r/GlobalOffensive/comments/dybzmg/operation_s...,self.GlobalOffensive,,1122,964,false,false,false,false,0,listing,NaN
1,"[, thing, id-t3_dyv3n3, linkflair, linkflair-d...",thing_t3_dyv3n3,click_thing(this),t3_dyv3n3,link,0,all_ads,CSGOMatchThreads,t2_mwknk,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1574216163000,/r/GlobalOffensive/comments/dyv3n3/csgo_asia_c...,/r/GlobalOffensive/comments/dyv3n3/csgo_asia_c...,self.GlobalOffensive,,36,34,false,false,false,false,0,listing,NaN
2,"[, thing, id-t3_dyw02a, linkflair, linkflair-d...",thing_t3_dyw02a,click_thing(this),t3_dyw02a,link,0,all_ads,oknani,t2_19o6bnax,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1574220386000,https://twitter.com/EliGE/status/1196922724980...,/r/GlobalOffensive/comments/dyw02a/elige_on_tw...,twitter.com,1,230,3002,false,false,false,false,0,listing,NaN
3,"[, thing, id-t3_dyyhu4, linkflair, linkflair-u...",thing_t3_dyyhu4,click_thing(this),t3_dyyhu4,link,0,all_ads,naathhh,t2_62oho,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1574233938000,https://i.redd.it/25mni5n5ksz31.png,/r/GlobalOffensive/comments/dyyhu4/radar_direc...,i.redd.it,2,22,653,false,false,false,false,0,listing,NaN


In [0]:
# dfposts["data-permalink"][0:1]

In [0]:
# for postlink in dfposts["data-permalink"][0:1]:
#   print("LINK:", postlink)
#   d.get(HTTPS + postlink)
#   print(d.page_source) # results

In [84]:
# Get all comments from every posts, using recursive scrape-store to differentiate between childs & parents.
# for postlink in dfposts["data-permalink"]:

for postlink in dfposts["data-permalink"][0:1]:
  d.get(HTTPS + postlink)
  print(postlink)
  coms = []

  # Get the raw text page source
  html = d.page_source

  # Find the div containing the comments list, then get the comments unrecursively.
  htcoms = soup(html, "html.parser").select("div.nestedlisting")[0].find_all('div', class_="comment", recursive=False)

  # Iterate through every comment found
  print(len(htcoms))
  for htcom in htcoms:
    # Get comment & vote count
    if com is not None:
      text = htcom.find('div', class_="usertext-body").text
      vote = htcom.find('span', class_="unvoted").text

      # Deleted comments. Also automatically None if hidden
      if vote == '':
        vote = None
      if vote != None:
        vote = int(vote[:-6])

      attrs = htcom.attrs # Get everything else (in attribute)

      attrs.update({'comment':text, "vote":vote})
      coms.append(attrs)

dfcoms = pd.DataFrame(coms)
dfcoms[0:4]

/r/GlobalOffensive/comments/dybzmg/operation_shattered_web_bugs_and_general/
70


,class,id,onclick,data-fullname,data-type,data-gildings,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-author,data-author-fullname,data-replies,data-permalink,comment,vote
0,"[, thing, id-t1_f806q78, noncollapsed, comment, ]",thing_t1_f806q78,click_thing(this),t1_f806q78,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,iiSoleHorizons,t2_i8owzf0,33,/r/GlobalOffensive/comments/dybzmg/operation_s...,"EDIT: Since it seems this is a large issue, he...",196
1,"[, thing, id-t1_f80cg64, noncollapsed, comment, ]",thing_t1_f80cg64,click_thing(this),t1_f80cg64,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,freaky112,t2_u09bx,3,/r/GlobalOffensive/comments/dybzmg/operation_s...,"If you buy the operation, the coin is automati...",158
2,"[, thing, id-t1_f806eo9, noncollapsed, comment, ]",thing_t1_f806eo9,click_thing(this),t1_f806eo9,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,Stigasaurus_Rexx,t2_ixpfe,4,/r/GlobalOffensive/comments/dybzmg/operation_s...,Can't believe it. After so long we finally get...,302
3,"[, thing, id-t1_f806w5c, noncollapsed, comment, ]",thing_t1_f806w5c,click_thing(this),t1_f806w5c,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,Hallsway,t2_a7w80,2,/r/GlobalOffensive/comments/dybzmg/operation_s...,So anyone found out if cobblestone items are b...,128


In [57]:
# Get all comments from every posts, unable to differentiate between childs & parents.
# for postlink in dfposts["data-permalink"]:
for postlink in dfposts["data-permalink"][0:1]:
  d.get(HTTPS + postlink)
  print(postlink)
  coms = []

  # Match all comments: div elements having both "thing" and "comment"
  # wdcoms = d.find_elements_by_css_selector("div")
  # Match all comments: div elements having "comment"
  wdcoms = d.find_elements_by_css_selector("div.thing.comment")

  # Iterate through every comment found
  for wdcom in wdcoms:

    # Get comment & vote count
    text = wdcom.find_element_by_css_selector("div.usertext-body").text
    try:
      vote = wdcom.find_element_by_css_selector("span.score.unvoted").text
    except NoSuchElementException:
      # Comments with temporarily hidden vote count
      vote = None

    # Deleted comments
    if vote == '':
      vote = None
    if vote != None:
      vote = int(vote[:-6])

    # Get everything else (in attribute)
    html = wdcom.get_attribute("outerHTML")
    attrs = soup(html, "html.parser").div.attrs

    attrs.update({'comment':text, 'vote':vote})
    # pprint(attrs)
    coms.append(attrs)

dfcoms = pd.DataFrame(coms)
dfcoms[0:4]

/r/GlobalOffensive/comments/dybzmg/operation_shattered_web_bugs_and_general/


,class,id,onclick,data-fullname,data-type,data-gildings,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-author,data-author-fullname,data-replies,data-permalink,comment,vote
0,"[, thing, id-t1_f806q78, noncollapsed, comment, ]",thing_t1_f806q78,click_thing(this),t1_f806q78,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,iiSoleHorizons,t2_i8owzf0,33,/r/GlobalOffensive/comments/dybzmg/operation_s...,"EDIT: Since it seems this is a large issue, he...",196.0
1,"[, thing, id-t1_f8089q7, noncollapsed, comment, ]",thing_t1_f8089q7,click_thing(this),t1_f8089q7,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,YaVs,t2_7cqfw,4,/r/GlobalOffensive/comments/dybzmg/operation_s...,Exactly the same for me :(\nedit : (Mac user),62.0
2,"[, thing, id-t1_f80hds2, noncollapsed, comment, ]",thing_t1_f80hds2,click_thing(this),t1_f80hds2,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,fenner-,t2_q5t0f9v,2,/r/GlobalOffensive/comments/dybzmg/operation_s...,yup exact same. am also on mac. real frustrati...,30.0
3,"[, thing, id-t1_f80wow9, noncollapsed, comment, ]",thing_t1_f80wow9,click_thing(this),t1_f80wow9,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,barbiekewt,t2_1oqiu00,6,/r/GlobalOffensive/comments/dybzmg/operation_s...,Same boat here and I have found a temporary so...,21.0


In [0]:
print(dfcoms)